In [1]:
#Preprocessing
import sys
sys.path = ['/nfs/gns/homes/willj/anaconda3/envs/GTEx/lib/python3.5/site-packages'] + sys.path
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
import pandas as pd
import requests
import openslide
from openslide.deepzoom import DeepZoomGenerator
from openslide import open_slide
import math
import pdb
import os
%matplotlib inline
GTEx_directory = '/hps/nobackup/research/stegle/users/willj/GTEx'
import mahotas
import sys
import argparse

In [2]:
import gzip
import pandas as pd
import numpy as np
import h5py
import os
import requests
import argparse

def get_donor_IDs(IDlist):
    return [str(x).split('-')[1] for x in IDlist]

def download_tissues(random_tissue_ID_lookup):
    for t4 in list(random_tissue_ID_lookup.keys()):
        random_image_IDs = random_tissue_ID_lookup[t4]
        try:
            os.mkdir(os.path.join('data',t4))
        except:
            print('Directory exists')
        print('Downloading tissue: {}'.format(t4))
        download(random_image_IDs, os.path.join('data',t4))


def openList(samples_filename):
    with open(samples_filename) as f:
        samples_list = f.read().splitlines()
    return(samples_list)

In [5]:
def download(samples_list, directory):
    print("Downloading images...")
    session = requests.session()

    # Do download
    for i in range(0, len(samples_list)):

        output_filename = os.path.join(directory,samples_list[i]+".svs")
        if not os.path.isfile(output_filename):
            print("Downloading "+str(i+1)+"/"+str(len(samples_list))+": "+samples_list[i])
            with open(output_filename, 'wb') as outfile:
                URL = "https://brd.nci.nih.gov/brd/imagedownload/" + str(samples_list[i])
                response = session.get(URL)
                if not response.ok:
                    print("Could not download")
                    raise Exception
                else:
                    outfile.write(response.content)
        else:
            print(str(i+1)+"/"+str(len(samples_list))+": "+samples_list[i] +" already Downloaded")
    return


genotypes_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/GenotypeFiles/phg000520.v2.GTEx_MidPoint_Imputation.genotype-calls-vcf.c1/parse_data/GTEx_Analysis_20150112_OMNI_2.5M_5M_450Indiv_chr1to22_genot_imput_info04_maf01_HWEp1E6_ConstrVarIDs_all_chrom_filered_maf_subset_individuals_44_tissues.hdf5'
expression_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/ExpressionFiles/phe000006.v2.GTEx_RNAseq.expression-data-matrixfmt.c1/parse_data/44_tissues/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_gene_rpkm_*_normalised_without_inverse_gene_expression.txt'
phenotype_filepath = '/nfs/research2/stegle/stegle_secure/GTEx/download/49139/PhenoGenotypeFiles/RootStudyConsentSet_phs000424.GTEx.v6.p1.c1.GRU/PhenotypeFiles/phs000424.v6.pht002743.v6.p1.c1.GTEx_Sample_Attributes.GRU.txt.gz'
GTEx_directory = '/hps/nobackup/research/stegle/users/willj/GTEx'

#Phenotype
with gzip.open(phenotype_filepath, 'rb') as f:
    phenotype_df = pd.DataFrame([str(x, 'utf-8').split('\t') for x in f.read().splitlines() if not str(x, 'utf-8').startswith('#')][1:])
    phenotype_df.columns = phenotype_df.iloc[0]
    phenotype_df = phenotype_df[1:]
    phenotype_IDs = ['-'.join(x.split('-')[0:3]) for x in list(phenotype_df['SAMPID']) if x.startswith('GTEX')]
    donor_phenotype_IDs = get_donor_IDs(phenotype_IDs)
    unique_donor_phenotype_IDs = np.unique(donor_phenotype_IDs)
    # 11983 phenotype IDs.
    # 571 unique phenotype donors

#Genotype
with h5py.File(genotypes_filepath,'r') as f:
    genotype_IDs = f['genotype']['row_header']['sample_ID']
    donor_genotype_IDs = get_donor_IDs(genotype_IDs)
    unique_donor_genotype_IDs = np.unique(donor_genotype_IDs)
    # 449 genotypes
    # 449 unique genotype donors

#Images
with open(GTEx_directory + '/' + 'data/aperio-downloader/gtex-sampid.txt','r') as f:
    image_IDs = f.read().splitlines()
    donor_image_IDs = get_donor_IDs(image_IDs)
    unique_donor_image_IDs = np.unique(donor_image_IDs)
    # 22642 images
    # 868 unique image donors

tissue_sample_counts = list(zip(phenotype_df['SMTSD'].value_counts().index, phenotype_df['SMTSD'].value_counts().tolist()))

In [6]:
tissue_sample_counts = list(zip(phenotype_df['SMTSD'].value_counts().index, phenotype_df['SMTSD'].value_counts().tolist()))

# unique image donors U unique genotype donors = 447
len(set(unique_donor_image_IDs).intersection(unique_donor_genotype_IDs))
# unique image donors U unique phenotype donors = 567
len(set(unique_donor_image_IDs).intersection(unique_donor_phenotype_IDs))
# unique genotype donors U unique phenotype donors = 449
len(set(unique_donor_genotype_IDs).intersection(unique_donor_phenotype_IDs))
# # unique genotype donors U unique phenotype donors U unique image donors  = 447
len(set(unique_donor_genotype_IDs).intersection(unique_donor_phenotype_IDs).intersection(set(unique_donor_image_IDs)))

# Define tissue types of interest
tissue_types = ['Lung', 'Artery - Tibial', 'Heart - Left Ventricle', 'Breast - Mammary Tissue', 'Brain - Cerebellum', 'Pancreas', 'Testis', 'Liver', 'Ovary', 'Stomach']

# Lookup tissues IDs from phenotype_df
tissue_IDs_set = [x for x in [list(phenotype_df[phenotype_df['SMTSD'] == tissue]['SAMPID']) for tissue in tissue_types]]
tissue_image_IDs = [list(set(['-'.join(x.split('-')[:3]) for x in tissue_IDs]).intersection(set(image_IDs))) for tissue_IDs in tissue_IDs_set]
tissue_ID_lookup = dict((tissue_types[i], tissue_image_IDs[i]) for i in range(len(tissue_image_IDs)))

In [15]:
with open(GTEx_directory + '/textfiles/ID_tissue_list.txt', 'w') as f:
    for t in tissue_ID_lookup.keys():
        for ID in tissue_ID_lookup[t]:
            print (ID + ' ' + t + '\n')
            f.write(ID + ' ' + t + '\n')
            

GTEX-R55G-0826 Lung

GTEX-145MF-0726 Lung

GTEX-R55D-0926 Lung

GTEX-12KS4-0726 Lung

GTEX-P4QS-0526 Lung

GTEX-ZYWO-0826 Lung

GTEX-P78B-0926 Lung

GTEX-QMRM-0826 Lung

GTEX-ZY6K-0326 Lung

GTEX-XUJ4-1426 Lung

GTEX-ZQG8-0326 Lung

GTEX-14A6H-0526 Lung

GTEX-ZTTD-1126 Lung

GTEX-NL3G-0526 Lung

GTEX-11PRG-0926 Lung

GTEX-131XH-0426 Lung

GTEX-ZPU1-0926 Lung

GTEX-13FTZ-0526 Lung

GTEX-ZC5H-0926 Lung

GTEX-PVOW-1026 Lung

GTEX-WYBS-1126 Lung

GTEX-R55C-0526 Lung

GTEX-13NYS-1626 Lung

GTEX-OIZH-0526 Lung

GTEX-145LT-0326 Lung

GTEX-12WSN-0626 Lung

GTEX-12ZZW-0926 Lung

GTEX-WFON-0426 Lung

GTEX-13O61-0726 Lung

GTEX-13VXU-2726 Lung

GTEX-1399S-1726 Lung

GTEX-QDVN-0726 Lung

GTEX-148VI-0226 Lung

GTEX-ZLWG-0626 Lung

GTEX-X4LF-0526 Lung

GTEX-144GM-0126 Lung

GTEX-13N1W-0726 Lung

GTEX-WK11-0526 Lung

GTEX-111YS-0626 Lung

GTEX-U8T8-2226 Lung

GTEX-WFG7-0526 Lung

GTEX-13QBU-0726 Lung

GTEX-TMMY-0926 Lung

GTEX-VJYA-0326 Lung

GTEX-PSDG-1126 Lung

GTEX-OXRL-0526 Lung

GTEX-XYKS-0526 L